# hid_dataset_3_0 invalid records

Шаг apply начал падать (в декабре 2022) с сообщениями типа
```s

Caused by: org.apache.spark.api.python.PythonException: Traceback (most recent call last):
...
  File "/prj/adapters/apply.py", line 269, in _transform_rows
  File "/prj/adapters/model/base.py", line 313, in apply
    columns = self.predict_adapter.predict(self.estimator, dataset)
  File "/prj/adapters/predict.py", line 74, in predict
    predicts = dict(predict_method(dataset.x, *steps, **predict_params))
  File "/prjcore/predictors/pipeline.py", line 215, in yield_from_steps
    Xt = step.transform(Xt, **trf_params)
  File "/prjcore/transformers/preprocessing.py", line 207, in transform
    return su.check_array(X_res, accept_sparse=sparse_format)
  File "/sklearn/utils/validation.py", line 517, in check_array
    accept_large_sparse=accept_large_sparse)
  File "/sklearn/utils/validation.py", line 350, in _ensure_sparse_format
    allow_nan=force_all_finite == 'allow-nan')
  File "/sklearn/utils/validation.py", line 56, in _assert_all_finite
    raise ValueError(msg_err.format(type_err, X.dtype))
ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

```

Метом джойна left_anti витрины с размеченной выборкой получил 100 записей, сохранил их в паркет
`hdfs:/user/vlk/TRG-80367-joiner_features/hid_dataset_3_0/2022-12-03/skip-clal_adhoc_audience-apply`

Теперь надо провести анализ этой вырезки и понять, что не так.

In [ ]:
import pprint
from pprint import pformat

import os
import datetime

from operator import and_
from collections import defaultdict

import six
import luigi
import pyspark
import pyspark.sql.functions as sqlfn

import json
import itertools as it

from pyspark.sql.types import MapType, ArrayType, FloatType, StringType, NumericType

if six.PY3:
    from functools import reduce  # make flake8 happy

In [ ]:
# pprint.pprint(dict(os.environ), width=1)
def log(obj, msg=""):
    if msg: print(msg)
    print("type: {}\ndata: {}".format(type(obj), pformat(obj, indent=1, width=1)))

log(os.environ, "os.environ")
print()
log(dict(os.environ), "dict(os.environ)")

In [ ]:
import os
import sys

from pyspark.sql import SparkSession, SQLContext

# Pack executable prj conda environment into zip
TMP_ENV_BASEDIR = "tmpenv"  # Reserved directory to store environment archive
env_dir = os.path.dirname(os.path.dirname(sys.executable))
env_name = os.path.basename(env_dir)
env_archive = "{basedir}/{env}.zip#{basedir}".format(basedir=TMP_ENV_BASEDIR, env=env_name)
os.environ["PYSPARK_PYTHON"] = "{}/{}/bin/python".format(TMP_ENV_BASEDIR, env_name)

# you need this only first time!
# !rm -rf {TMP_ENV_BASEDIR} && mkdir {TMP_ENV_BASEDIR} && cd {TMP_ENV_BASEDIR} && rsync -a {env_dir} . && zip -rq {env_name}.zip {env_name}

# b.config("spark.yarn.dist.archives", env_archive)
log(env_archive)

In [ ]:
# Create Spark session with prj conda environment and JVM extensions
# `spark-submit ... --driver-java-options "-Dlog4j.configuration=file:/home/vlk/driver_log4j.properties"`
# spark.driver.extraJavaOptions

queue = "root.priority"

# "spark.driver.extraJavaOptions", "-Xss10M"
# catalyst SO while building parts. filter expression

# 200 GB of data
# sssp = (200 * 4) * 2 * 2 * 4
sssp = 12

spark = (
SparkSession.builder
    .master("yarn-client")
    .appName("TRG-92508-test-ipynb")
    .config("spark.yarn.queue", queue)
    .config("spark.sql.shuffle.partitions", sssp)
    .config("spark.executor.instances", "2")
    .config("spark.executor.cores", "6")
    .config("spark.executor.memory", "6G")
    .config("spark.executor.memoryOverhead", "6G")
    .config("spark.driver.memory", "4G")
    .config("spark.driver.maxResultSize", "2G")
    .config("spark.driver.extraJavaOptions", "-Dlog4j.configuration=file:/home/vlk/driver2_log4j.properties")
    .config("spark.speculation", "true")
    .config("spark.dynamicAllocation.enabled", "true")
    .config("spark.dynamicAllocation.minExecutors", "2")
    .config("spark.dynamicAllocation.maxExecutors", "256")
    .config("spark.dynamicAllocation.executorIdleTimeout", "300s")
    .config("spark.network.timeout", "800s")
    .config("spark.reducer.maxReqsInFlight", "10")
    .config("spark.shuffle.io.retryWait", "60s")
    .config("spark.shuffle.io.maxRetries", "10")
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
    .config("spark.kryoserializer.buffer.max", "1024m")
    .config("spark.hadoop.hive.exec.dynamic.partition", "true")
    .config("spark.hadoop.hive.exec.dynamic.partition.mode", "nonstrict")
    .config("spark.hadoop.hive.exec.max.dynamic.partitions", "1000000")
    .config("spark.hadoop.hive.exec.max.dynamic.partitions.pernode", "100000")
    .config("spark.hadoop.hive.metastore.client.socket.timeout", "60s")
    .config("spark.ui.enabled", "true")
    .config("spark.sql.sources.partitionColumnTypeInference.enabled", "false")
    .config("spark.yarn.dist.archives", env_archive)
    .getOrCreate()
)
# .config("spark.driver.extraJavaOptions", "-Xss10M -Dlog4j.configuration=file:/home/vlk/driver_log4j.properties")
# .config("spark.jars", "hdfs:/lib/dm/prj-transformers-assembly-dev-1.5.1.jar")

sql_ctx = SQLContext(spark.sparkContext)
(spark, sql_ctx)

In [ ]:
# end of env. setup

In [ ]:
import os
import numpy as np
import sklearn.utils as su

from boltons.iterutils import chunked_iter

from pprint import pformat

import luigi

import pyspark.sql.functions as sqlfn

from pyspark.storagelevel import StorageLevel

from pyspark.sql import Row, SQLContext, SparkSession
from pyspark.sql import DataFrame, SQLContext
from pyspark.sql.types import (
    MapType, ArrayType, FloatType, DoubleType, StringType, StructType, IntegralType, IntegerType
)
from pyspark.sql.utils import CapturedException
from pyspark.ml.wrapper import JavaWrapper

from luigi.contrib.hdfs import HdfsTarget

from prj.apps.utils.common import add_days
from prj.apps.utils.common.hive import format_table, select_clause
from prj.apps.utils.control.client.logs import ControlLoggingMixin
from prj.apps.utils.common.external_program import AvoidLuigiFlatTaskRunner

from prj.apps.utils.common import unfreeze_json_param
from prj.apps.utils.common.fs import HdfsClient
from prj.apps.utils.common.hive import FindPartitionsEngine
from prj.apps.utils.common.luigix import HiveTableSchemaTarget

from prj.apps.utils.common.hive import select_clause
from prj.apps.utils.common.spark import CustomUDFLibrary, insert_into_hive
from prj.apps.utils.common.luigix import HiveExternalTask, HiveGenericTarget
from prj.apps.utils.control.luigix import ControlApp, ControlDynamicOutputPySparkTask

from prj.common.hive import HiveMetastoreClient, HiveThriftSASLContext

from prj.apps.utils.common.spark import PySparkRows2LearningData
from prj.adapters.datafile import (
    LabelNamesAdapter, LearningDataAdapter, GroupedFeaturesAdapter, AbstractDataFileAdapter
)
from prj.adapters.dataset import LearnsetAdapter, AbstractDatasetAdapter
from prj.adapters.model.oclf import OrdClfLGBMRegressor
from prjcore.utils.io import LearningData


In [ ]:
# CustomUDFLibrary(spark, "hdfs:/lib/dm/prj-transformers-assembly-dev-1.5.1.jar").register_all_udf()

In [ ]:
def show(df, message="dataframe", nlines=20, truncate=False, heavy=True):
    if not heavy: print("\n{}, rows: {}:".format(message, df.count()))
    df.printSchema()
    if not heavy: df.show(nlines, truncate)
    return df

In [ ]:
hid_dataset_100_path = (
    "hdfs:/user/vlk/TRG-80367-joiner_features/"
    "hid_dataset_3_0/2022-12-03/skip-clal_adhoc_audience-apply"
)

In [ ]:
features_df = spark.read.parquet(hid_dataset_100_path).persist(StorageLevel.MEMORY_ONLY)

In [ ]:
show(features_df, heavy=False)

In [ ]:
features_rows = features_df.collect()

In [ ]:
log(features_rows[44])  # list[Row]
# uid=u'7759301908035527299'
# tp_device_stats=[3.0750763416290283, 0.0, inf]

# found: invalid value: inf in array of float values

In [ ]:
orig = features_rows[44]
# log(orig.asDict())
tp_device_stats = orig["tp_device_stats"]
log(tp_device_stats, "bad")
tp_device_stats = tp_device_stats[:2] + [0.0]
log(tp_device_stats, "fixed")

fixed = orig.asDict()
fixed["tp_device_stats"] = tp_device_stats
fixedRow = Row(**fixed)
log(fixedRow, "fixed row")

features_rows[44] = fixedRow
log(features_rows[44])

In [ ]:
# x = rows_transformer.transform(chunk).x

In [ ]:
t = """
rm -v ./grouped_features.json
hdfs dfs -copyToLocal /user/vlk/tmp/personal_income__HID/grouped_features.json
gdfs put ~/grouped_features.json /user/vlk/
fshare /user/vlk/grouped_features.json
"""
grouped_features_adapter = GroupedFeaturesAdapter()
grouped_features = grouped_features_adapter.load("/home/vlk/").data
log(grouped_features)

In [ ]:
grouped_features_list = []
grouped_features_list.append(grouped_features)

grouped_features_all, model_column_slices = grouped_features_adapter.merge(*grouped_features_list)

rows_transformer = PySparkRows2LearningData(
    schema=features_df.schema, 
    numeric_type=np.float32, 
    features=grouped_features_all
)

log(rows_transformer)

In [ ]:
# for chunk in chunked_iter(rows, bc.value["batch_size"])
chunk = features_rows
x = rows_transformer.transform(chunk).x
log(x)

In [ ]:
# return su.check_array(X_res, accept_sparse=sparse_format)

x_checked = su.check_array(x, accept_sparse="csc", force_all_finite=True)

In [ ]:
# dataset = construct(bc.value["dataset_adapter"])
dataset = LearnsetAdapter()

# prj.adapters.model.oclf.OrdClfLGBMRegressor
model_adapter = OrdClfLGBMRegressor().load("/home/vlk/")

In [ ]:
chunk_num = 0
for chunk in chunked_iter(features_rows, 1):
    chunk_num += 1
    if chunk_num < 45:
        continue

    x = rows_transformer.transform(chunk).x
    log((chunk_num, x, x.data), "current row")

    dataset.set_data(LearningData(x=x[:, model_column_slices[0]]))
    prediction_dicts = model_adapter.apply(dataset)

    log((prediction_dicts, list(prediction_dicts)), "prediction")

#     log(np.isfinite(x.data))
#     x_checked = su.check_array(x, accept_sparse="csc", force_all_finite=True)

In [ ]:
spark.stop()